<a href="https://colab.research.google.com/github/AlbertoAlaldu/SRP/blob/main/PRS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
"""
Análisis SRP/PVP con EV-02C REAL (simulador) en los tres regímenes RE/RH/RC.

Flujo:
  - Carga un YAML base (el de tu artículo APP, EV-02C).
  - Define un parámetro de diseño escalar p que escala phi_min (daño por tick).
  - Para cada régimen (RE, RH, RC) y cada p:
      - Modifica la config en memoria.
      - Llama al simulador EV-02C real.
      - Extrae:
          * W = viabilidad (tiempo de vida medio por agente)
          * ρ̄ = reducción sistémica efectiva (promedio temporal)
          * G_eff, C_eff = términos efectivos de ganancia y costo.
  - Genera:
      * Tabla por régimen: (p, ρ̄, W, G_eff, C_eff)
      * Curva W vs ρ̄
      * Curvas G_eff(ρ̄), C_eff(ρ̄)

IMPORTANTE:
  - Completa las dos funciones marcadas con TODO(Ing. Alberto):
      1) run_ev02_simulation_once(...)
      2) compute_effective_rho_G_C(...)
"""

import copy
import yaml
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Dict, Tuple, List


# ============================================================
# 1. CONFIGURACIÓN GENERAL
# ============================================================

# Ruta al YAML base de EV-02C que usaste en el artículo
# (guarda el YAML que pegaste como, por ejemplo, "config_ev02_base.yaml")
CONFIG_PATH = Path("config_ev02_base.yaml")

# Arquitectura y regímenes a analizar
ARCH = "EV-02C"
REGIMES = ["RE", "RH", "RC"]

# Política para estudiar SRP (APP-Fixed, como acordamos)
POLICY = "APP-Fixed"

# Barrido de parámetro de diseño p
# p escala phi_min en cada régimen: phi_min(p) = phi_min_base * (1 + p)
P_VALUES = np.linspace(-0.5, 0.5, 11)  # puedes ajustar el rango si hace falta

# Número de semillas / corridas independientes por punto (p, régimen)
N_SEEDS = 40   # para exploración; subir a 100+ para figuras finales

# Longitud máxima de simulación (ticks)
# Debe ser consistente con T en el YAML base (pero no crítico para el análisis)
T_MAX = 300

# Directorio de salida para tablas y figuras
OUT_DIR = Path("ev02_srp_results")
OUT_DIR.mkdir(exist_ok=True, parents=True)


# ============================================================
# 2. UTILIDADES PARA CARGAR Y MODIFICAR EL YAML
# ============================================================

def load_base_config(path: Path) -> Dict:
    """Carga el YAML base de EV-02C."""
    with path.open("r") as f:
        cfg = yaml.safe_load(f)
    return cfg


def build_config_for_run(
    base_cfg: Dict,
    regime: str,
    p: float,
    seed: int,
) -> Dict:
    """
    Toma el YAML base y construye una config para:
      - Arquitectura EV-02C
      - Régimen específico (RE, RH, RC)
      - Política APP-Fixed
      - Semilla dada
      - phi_min escalado por (1 + p)

    NO escribe nada a disco, solo devuelve un dict.
    """
    cfg = copy.deepcopy(base_cfg)

    # Sección 'general'
    if "general" not in cfg:
        raise ValueError("El YAML base debe tener sección 'general'.")

    cfg["general"]["architecture"] = ARCH
    cfg["general"]["regime"] = regime
    cfg["general"]["policy"] = POLICY
    cfg["general"]["seed"] = int(seed)

    # Sección de regímenes
    if "regimes" not in cfg:
        raise ValueError("El YAML base debe tener sección 'regimes'.")

    if regime not in cfg["regimes"]:
        raise ValueError(f"Régimen '{regime}' no encontrado en 'regimes' del YAML.")

    # Escalamos phi_min como parámetro de diseño p
    phi_base = cfg["regimes"][regime]["phi_min"]
    cfg["regimes"][regime]["phi_min"] = float(phi_base * (1.0 + p))

    # Puedes, si quieres, escalar otros parámetros aquí (k_rep, P_basal, etc.)
    # pero por ahora nos quedamos con phi_min como canal principal de "protección".

    return cfg


# ============================================================
# 3. CONECTOR AL SIMULADOR EV-02C REAL
# ============================================================

def run_ev02_simulation_once(cfg: Dict) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Llama a TU simulador real (simulation_app_base_v2_FINAL) una vez.

    Debe devolver:
      - df_ticks: DataFrame con series temporales agregadas (por tick) de los agentes
      - df_summary: DataFrame/Series con estadísticas de viabilidad (incluyendo lifespan)

    TODO(Ing. Alberto):
      - Sustituye el cuerpo de esta función por la llamada real a tu simulador.
      - Lo más probable: importas simulation_app_base_v2_FINAL y llamas algo tipo:
            df_ticks, df_summary = run_from_config_dict(cfg)
        o lo que ya estés usando en tus scripts.

    IMPORTANTE:
      - df_ticks debe contener columnas suficientes para reconstruir:
          * energía (gamma, o lo que uses)
          * daño (h)
          * ciclos C
          * biomasa B (si aplica)
      - df_summary debe tener (o permitir obtener) el tiempo de vida medio W
        (o bien por-agente y tú promedias).
    """
    # EJEMPLO de estructura (NO FUNCIONAL, reemplazar por tu código):
    # from simulation_app_base_v2_FINAL import run_from_cfg
    # df_ticks, df_summary = run_from_cfg(cfg)
    # return df_ticks, df_summary

    raise NotImplementedError(
        "Conecta aquí tu simulador EV-02C real (simulation_app_base_v2_FINAL)."
    )


# ============================================================
# 4. CÁLCULO DE ρ̄, G_eff y C_eff A PARTIR DE LAS SALIDAS
# ============================================================

def compute_effective_rho_G_C(
    df_ticks: pd.DataFrame,
    regime: str,
) -> Tuple[float, float, float]:
    """
    A partir de df_ticks calcula:
      - rho_eff: reducción sistémica media (escala [0, 1])
      - G_eff: término efectivo de ganancia
      - C_eff: término efectivo de costo

    TODO(Ing. Alberto):
      - Implementa aquí EXACTAMENTE la receta que usaste en el artículo
        para definir "reducción sistémica" y separar G y C (energía/ruido vs protección).

    Sugerencia de estructura:
      - Normalizas canales con parámetros de appendix_A y del régimen:
          * gamma / gamma_safe
          * h / h_max
          * C / C_max
          * B / B_min
      - Defines funciones de "gating" (0–1) para cada canal.
      - Combinas esos gates en una reducción ρ(t) ∈ [0,1], luego promedias en tiempo
        (p.ej. sobre los últimos 50–100 ticks).
      - G_eff y C_eff pueden definirse como:
          G_eff = promedio_t( G(ρ(t); canales) )
          C_eff = promedio_t( C(ρ(t); canales) )
        con la misma lógica del paper SRP.

    En esta plantilla dejo un esquema mínimo que TIENES que adaptar a tus columnas reales.
    """

    # --------- EJEMPLO GENÉRICO (ADAPTAR A TU DATAFRAME) ---------
    # Supongamos que df_ticks tiene columnas:
    #   'tick', 'gamma_mean', 'h_mean', 'C_mean', 'B_mean'
    # Ajusta estos nombres a los de tu simulador.
    col_gamma = "gamma_mean"
    col_h = "h_mean"
    col_C = "C_mean"
    col_B = "B_mean"  # si no hay biomasa, puedes omitir este canal

    if not all(c in df_ticks.columns for c in [col_gamma, col_h, col_C]):
        raise ValueError(
            "Adapta compute_effective_rho_G_C() a las columnas reales de df_ticks."
        )

    # Normalización básica usando parámetros estáticos del artículo
    # (puedes leerlos del YAML si quieres máxima fidelidad).
    gamma_safe = 0.60
    h_max = 1.0
    C_max = 55.0 if regime == "RC" else 100.0  # EJEMPLO basado en tu YAML
    B_min = 0.5

    gamma_norm = df_ticks[col_gamma] / (gamma_safe + 1e-9)
    h_norm = df_ticks[col_h] / (h_max + 1e-9)
    C_norm = df_ticks[col_C] / (C_max + 1e-9)

    if col_B in df_ticks.columns:
        B_norm = df_ticks[col_B] / (B_min + 1e-9)
    else:
        B_norm = None

    # EJEMPLO MUY SIMPLE de construcción de ρ(t):
    #  - mayor energía relativa = menos reducción (más apertura)
    #  - mayor daño relativo = más reducción (más cierre)
    #  - mayor ocupación de ciclos = más reducción
    #  - biomasa la puedes tratar como otro canal, o ignorarla si no aplica aquí.
    # OJO: ESTO ES SOLO UNA PLANTILLA, NO ES LA DEFINICIÓN DEL PAPER.
    rho_t = (
        0.25 * (1.0 - gamma_norm.clip(0.0, 1.0))
        + 0.25 * h_norm.clip(0.0, 1.0)
        + 0.25 * C_norm.clip(0.0, 1.0)
        + (0.25 * (1.0 - B_norm.clip(0.0, 1.0)) if B_norm is not None else 0.0)
    )

    # Aseguramos rango [0,1]
    rho_t = rho_t.clip(0.0, 1.0)

    # Para evitar incluir fase transitoria, podemos tomar último tercio de la simulación
    if "tick" in df_ticks.columns:
        ticks = df_ticks["tick"].to_numpy()
        t_cut = np.percentile(ticks, 66)
        mask = ticks >= t_cut
        rho_eff = float(rho_t[mask].mean())
    else:
        rho_eff = float(rho_t.mean())

    # Definición simple de G_eff y C_eff:
    #    G_eff ~ cuanto "acceso" efectivo queda (1 - rho)
    #    C_eff ~ cuanto "costo/protección" se está pagando (rho)
    # En el paper SRP tienes una definición más rica; sustitúyela aquí.
    G_eff = float((1.0 - rho_t).mean())
    C_eff = float(rho_t.mean())

    return rho_eff, G_eff, C_eff


# ============================================================
# 5. LOOP PRINCIPAL: BARRIDO DE p Y REGÍMENES
# ============================================================

def run_srp_analysis():
    base_cfg = load_base_config(CONFIG_PATH)

    all_results: List[pd.DataFrame] = []

    for regime in REGIMES:
        print(f"\n=== Régimen: {regime} ===")

        rows = []

        for p in P_VALUES:
            print(f"  p = {p:+.3f}")
            Ws = []
            rhos_eff = []
            Gs = []
            Cs = []

            for seed in range(N_SEEDS):
                cfg = build_config_for_run(base_cfg, regime, p, seed)
                df_ticks, df_summary = run_ev02_simulation_once(cfg)

                # ---- W (tiempo de vida medio) ----
                # Opciones:
                #   - df_summary tiene 'mean_lifespan' (directo)
                #   - o bien tiene una fila por agente con 'lifespan' y tú promedias
                if "mean_lifespan" in df_summary.columns:
                    W = float(df_summary["mean_lifespan"].iloc[0])
                elif "lifespan" in df_summary.columns:
                    W = float(df_summary["lifespan"].mean())
                else:
                    raise ValueError(
                        "df_summary debe tener 'mean_lifespan' o 'lifespan'. Ajusta aquí."
                    )

                # ---- ρ̄, G_eff, C_eff ----
                rho_eff, G_eff, C_eff = compute_effective_rho_G_C(df_ticks, regime)

                Ws.append(W)
                rhos_eff.append(rho_eff)
                Gs.append(G_eff)
                Cs.append(C_eff)

            # Agregamos por (regime, p)
            row = dict(
                regime=regime,
                p=float(p),
                rho_eff=float(np.mean(rhos_eff)),
                W=float(np.mean(Ws)),
                G_eff=float(np.mean(Gs)),
                C_eff=float(np.mean(Cs)),
                W_std=float(np.std(Ws)),
                rho_std=float(np.std(rhos_eff)),
            )
            rows.append(row)

        df_regime = pd.DataFrame(rows)
        all_results.append(df_regime)

        # Guardamos CSV por régimen
        out_csv = OUT_DIR / f"ev02_srp_{regime}.csv"
        df_regime.to_csv(out_csv, index=False)
        print(f"  -> Resultados guardados en {out_csv}")

        # También graficamos W vs rho, y G/C vs rho
        plot_viability_and_GC(df_regime, regime)

    # Tabla global
    df_all = pd.concat(all_results, ignore_index=True)
    df_all.to_csv(OUT_DIR / "ev02_srp_all_regimes.csv", index=False)
    print(f"\nResultados globales guardados en {OUT_DIR / 'ev02_srp_all_regimes.csv'}")


# ============================================================
# 6. FIGURAS
# ============================================================

def plot_viability_and_GC(df_regime: pd.DataFrame, regime: str):
    """Genera figuras W vs rho_eff y G/C vs rho_eff para un régimen."""
    df = df_regime.sort_values("rho_eff").reset_index(drop=True)

    rho = df["rho_eff"].to_numpy()
    W = df["W"].to_numpy()
    G = df["G_eff"].to_numpy()
    C = df["C_eff"].to_numpy()

    # 1) W vs rho
    plt.figure(figsize=(7, 4))
    plt.plot(rho, W, marker="o")
    plt.xlabel("ρ_eff (reducción sistémica efectiva)")
    plt.ylabel("W = tiempo de vida medio")
    plt.title(f"EV-02C ({regime}) - Curva de viabilidad vs ρ_eff")
    plt.grid(True)

    rho_star = float(rho[np.argmax(W)])
    W_star = float(W.max())
    plt.axvline(rho_star, linestyle="--")
    plt.scatter([rho_star], [W_star], marker="*", s=80,
                label=f"ρ* ≈ {rho_star:.3f}")
    plt.legend()

    out_png = OUT_DIR / f"ev02_{regime}_W_vs_rho.png"
    plt.savefig(out_png, dpi=150, bbox_inches="tight")
    plt.close()
    print(f"  -> Figura W vs rho guardada en {out_png}")

    # 2) G_eff y C_eff vs rho
    plt.figure(figsize=(7, 4))
    plt.plot(rho, G, marker="o", label="G_eff")
    plt.plot(rho, C, marker="s", label="C_eff")
    plt.xlabel("ρ_eff")
    plt.ylabel("Valor efectivo (normalizado)")
    plt.title(f"EV-02C ({regime}) - G_eff y C_eff vs ρ_eff")
    plt.grid(True)
    plt.legend()

    out_png2 = OUT_DIR / f"ev02_{regime}_G_C_vs_rho.png"
    plt.savefig(out_png2, dpi=150, bbox_inches="tight")
    plt.close()
    print(f"  -> Figura G/C vs rho guardada en {out_png2}")


# ============================================================
# 7. ENTRY POINT
# ============================================================

if __name__ == "__main__":
    run_srp_analysis()


FileNotFoundError: [Errno 2] No such file or directory: 'config_ev02_base.yaml'

In [ ]:
import copy
import numpy as np
import pandas as pd
import yaml

# Importamos tu simulador real
from simulation_app_base_v2_FINAL import Simulation

# Ruta al YAML base de EV-02C (el que pegaste)
BASE_CONFIG_PATH = "config_ev02c.yaml"  # ajusta el nombre si usaste otro


def load_base_config() -> dict:
    """Carga el YAML base de EV-02C."""
    with open(BASE_CONFIG_PATH, "r") as f:
        cfg = yaml.safe_load(f)
    return cfg


def build_config_for_p(base_cfg: dict, regime: str, p: float,
                       policy_name: str = "APP-Fixed",
                       architecture: str = "EV-02C",
                       n_agents: int = None,
                       T: int = None) -> dict:
    """
    Toma el YAML base y construye una config modificada para un valor de p.

    Aquí definimos el mapeo p -> F_E(p) = F_E_base * (1 + p),
    con p en [-0.5, 0.5]. Si quieres otra interpretación, solo
    cambia este bloque.
    """
    cfg = copy.deepcopy(base_cfg)

    # General
    cfg["general"]["architecture"] = architecture
    cfg["general"]["regime"] = regime
    cfg["general"]["policy"] = policy_name

    if n_agents is not None:
        cfg["general"]["N"] = n_agents
    if T is not None:
        cfg["general"]["T"] = T

    # Parámetros del régimen
    reg_cfg = cfg["regimes"][regime]
    F_E_base = reg_cfg["F_E"]

    # Mapeo de diseño: F_E(p) = F_E_base * (1 + p)
    # p = -0.5 -> 0.5*F_E; p = 0.5 -> 1.5*F_E
    F_E_eff = F_E_base * (1.0 + p)
    F_E_eff = max(0.0, F_E_eff)  # por si acaso

    reg_cfg["F_E"] = F_E_eff

    return cfg


def run_ev02_for_p(regime: str,
                   p: float,
                   seeds: int = 20,
                   n_agents: int = 40,
                   T: int = 300) -> float:
    """
    Corre el simulador EV-02C real para un régimen y un valor de p,
    promediando sobre varias semillas.

    Devuelve W(p) = vida media promedio.
    """
    base_cfg = load_base_config()

    lifetimes = []

    for seed in range(seeds):
        cfg_p = build_config_for_p(
            base_cfg,
            regime=regime,
            p=p,
            policy_name="APP-Fixed",
            architecture="EV-02C",
            n_agents=n_agents,
            T=T,
        )

        sim = Simulation(
            cfg=cfg_p,
            regime=regime,
            policy_name="APP-Fixed",
            architecture="EV-02C",
            n_agents=cfg_p["general"]["N"],
            T=cfg_p["general"]["T"],
            seed=seed,
            outputs_dir="outputs_srp_ev02",  # no se usa en run(), pero se requiere el argumento
            step_at=None,
            step_scale=1.5,
            step_dur=50,
        )

        df_ticks, summary, df_agents = sim.run()
        lifetimes.append(summary["mean_lifespan"])

    return float(np.mean(lifetimes))


def scan_p_for_regime(regime: str,
                      p_min: float = -0.5,
                      p_max: float = 0.5,
                      n_points: int = 11,
                      seeds: int = 20,
                      n_agents: int = 40,
                      T: int = 300) -> pd.DataFrame:
    """
    Barre valores de p en [p_min, p_max] para un régimen dado,
    y regresa un DataFrame con columnas: regime, p, W.
    """
    p_values = np.linspace(p_min, p_max, n_points)
    rows = []

    print(f"\n=== Régimen: {regime} ===")
    for p in p_values:
        W_p = run_ev02_for_p(
            regime=regime,
            p=p,
            seeds=seeds,
            n_agents=n_agents,
            T=T,
        )
        print(f"  p = {p:.3f}  -->  W(p) ≈ {W_p:.3f}")
        rows.append(dict(regime=regime, p=p, W=W_p))

    df = pd.DataFrame(rows)
    return df


def run_srp_ev02_full():
    """
    Corre el análisis SRP para EV-02C real en los tres regímenes RE/RH/RC
    y devuelve un único DataFrame concatenado.
    """
    dfs = []
    for regime in ("RE", "RH", "RC"):
        df_regime = scan_p_for_regime(regime)
        dfs.append(df_regime)

    df_all = pd.concat(dfs, ignore_index=True)
    print("\n=== Resultados agregados (primeras filas) ===")
    print(df_all.head())
    return df_all


# Si lo corres como script:
if __name__ == "__main__":
    df_all = run_srp_ev02_full()
    # Opcional: guardar a CSV
    df_all.to_csv("srp_ev02_W_vs_p.csv", index=False)



=== Régimen: RE ===
  p = -0.500  -->  W(p) ≈ 55.049
  p = -0.400  -->  W(p) ≈ 55.304
  p = -0.300  -->  W(p) ≈ 55.360
  p = -0.200  -->  W(p) ≈ 55.416
  p = -0.100  -->  W(p) ≈ 55.453
  p = 0.000  -->  W(p) ≈ 55.477
  p = 0.100  -->  W(p) ≈ 55.505
  p = 0.200  -->  W(p) ≈ 55.532
  p = 0.300  -->  W(p) ≈ 55.554
  p = 0.400  -->  W(p) ≈ 55.576
  p = 0.500  -->  W(p) ≈ 55.600

=== Régimen: RH ===
  p = -0.500  -->  W(p) ≈ 26.719
  p = -0.400  -->  W(p) ≈ 26.721
  p = -0.300  -->  W(p) ≈ 26.724
  p = -0.200  -->  W(p) ≈ 26.745
  p = -0.100  -->  W(p) ≈ 26.754
  p = 0.000  -->  W(p) ≈ 26.755
  p = 0.100  -->  W(p) ≈ 26.755
  p = 0.200  -->  W(p) ≈ 26.756
  p = 0.300  -->  W(p) ≈ 26.772
  p = 0.400  -->  W(p) ≈ 26.796
  p = 0.500  -->  W(p) ≈ 26.815

=== Régimen: RC ===
  p = -0.500  -->  W(p) ≈ 55.000
  p = -0.400  -->  W(p) ≈ 55.000
  p = -0.300  -->  W(p) ≈ 55.000
  p = -0.200  -->  W(p) ≈ 55.000
  p = -0.100  -->  W(p) ≈ 55.000
  p = 0.000  -->  W(p) ≈ 55.000
  p = 0.100  -->  W(p) ≈ 5

In [7]:
# srp_ev02c_real.py
#
# Análisis tipo SRP usando el EV-02C REAL del simulador
# (simulation_app_base_v2_FINAL.py) variando F_E con un parámetro p.
#
# W(p)     = mean_lifespan (tiempo de vida medio)
# G_eff(p) = mean_V_tail   (viabilidad interna promedio en cola)
# C_eff(p) = cv_ps         (costo: variabilidad relativa de E_env)
#
# Usa APP-Fixed como política, arquitecturas EV-02C, regímenes RE/RH/RC.

import copy
import os
from typing import Dict, List

import numpy as np
import pandas as pd
import yaml
import matplotlib.pyplot as plt

# IMPORTA TU SIMULADOR REAL
from simulation_app_base_v2_FINAL import Simulation


# ==============================
# 1. Carga de configuración base
# ==============================

CONFIG_PATH = "config_ev02c.yaml"  # <-- CAMBIA ESTO AL NOMBRE REAL DE TU YAML


def load_base_cfg(path: str) -> Dict:
    with open(path, "r") as f:
        cfg = yaml.safe_load(f)
    return cfg


# ============================================
# 2. Correr una simulación EV-02C para (regime, p)
# ============================================

def run_ev02c_once(
    base_cfg: Dict,
    regime: str,
    p: float,
    seed: int,
    n_agents: int = None,
    T: int = None,
    outputs_dir: str = "outputs_srp_ev02c",
) -> Dict:
    """
    Corre una simulación EV-02C real para un régimen y un valor de p.

    p escala F_E así:
        F_E(p) = F_E0 * (1 + p)

    Devuelve el diccionario 'summary' de Simulation.run(),
    extendido con campos regime y p.
    """
    cfg = copy.deepcopy(base_cfg)

    # --- General: forzamos EV-02C + régimen + APP-Fixed ---
    cfg["general"]["architecture"] = "EV-02C"
    cfg["general"]["regime"] = regime
    cfg["general"]["policy"] = "APP-Fixed"

    if n_agents is not None:
        cfg["general"]["N"] = n_agents
    if T is not None:
        cfg["general"]["T"] = T

    # --- Escalamos F_E para este régimen según p ---
    # F_E(p) = F_E0 * (1 + p), con p en [-0.5, 0.5] normalmente.
    F0 = cfg["regimes"][regime]["F_E"]
    cfg["regimes"][regime]["F_E"] = F0 * (1.0 + p)

    # Creamos carpeta de salida si no existe
    os.makedirs(outputs_dir, exist_ok=True)

    sim = Simulation(
        cfg=cfg,
        regime=regime,
        policy_name="APP-Fixed",
        architecture="EV-02C",
        n_agents=cfg["general"]["N"],
        T=cfg["general"]["T"],
        seed=seed,
        outputs_dir=outputs_dir,
        step_at=None,         # sin perturbación AIM-3
        step_scale=1.5,
        step_dur=50,
    )

    df_ticks, summary, df_agents = sim.run()

    # Enriquecemos el summary con info de p y régimen
    summary["regime"] = regime
    summary["p"] = p
    summary["seed"] = seed

    return summary


# =======================================
# 3. Promediar sobre semillas para cada p
# =======================================

def aggregate_over_seeds(
    base_cfg: Dict,
    regime: str,
    p_values: List[float],
    n_seeds: int = 30,
    n_agents: int = None,
    T: int = None,
) -> pd.DataFrame:
    """
    Para un régimen dado, barre p_values y promedia sobre n_seeds semillas.

    Devuelve DataFrame con columnas:
      regime, p, W, G_eff, C_eff
    """
    rows = []

    # E_cap para este régimen (para normalizar el costo)
    E_cap = base_cfg["regimes"][regime]["E_cap"]

    for p in p_values:
        summaries = []
        for seed in range(n_seeds):
            summary = run_ev02c_once(
                base_cfg=base_cfg,
                regime=regime,
                p=p,
                seed=seed,
                n_agents=n_agents,
                T=T,
                outputs_dir=f"outputs_srp_ev02c_{regime}",
            )
            summaries.append(summary)

        df = pd.DataFrame(summaries)

        # W(p): tiempo de vida medio
        W = df["mean_lifespan"].mean()

        # G_eff(p): viabilidad interna promedio en cola
        if "mean_V_tail" in df.columns:
            G_eff = df["mean_V_tail"].mean()
        else:
            G_eff = np.nan

        # C_eff(p): costo alternativo = var_ps normalizada por E_cap^2
        if "var_ps" in df.columns:
            C_eff = df["var_ps"].mean() / (E_cap ** 2)
        else:
            C_eff = np.nan

        rows.append(
            dict(
                regime=regime,
                p=p,
                W=W,
                G_eff=G_eff,
                C_eff=C_eff,
            )
        )

    return pd.DataFrame(rows)



# ==========================
# 4. Graficar W(p) por régimen
# ==========================

def plot_W_vs_p(df_results: pd.DataFrame, out_path: str = None):
    regimes = sorted(df_results["regime"].unique())
    plt.figure(figsize=(8, 5))

    for reg in regimes:
        sub = df_results[df_results["regime"] == reg].sort_values("p")
        plt.plot(sub["p"], sub["W"], marker="o", label=f"{reg}")

    plt.axvline(0.0, color="gray", linestyle="--", alpha=0.5)
    plt.xlabel("p (escala relativa de F_E)")
    plt.ylabel("W(p) = tiempo de vida medio")
    plt.title("Curvas W(p) para EV-02C real (APP-Fixed)")
    plt.legend()
    plt.grid(True)

    if out_path is not None:
        plt.savefig(out_path, dpi=150, bbox_inches="tight")
    else:
        plt.show()

    plt.close()


# ==========
# 5. Main
# ==========

def main():
    base_cfg = load_base_cfg(CONFIG_PATH)

    # Parámetros globales
    # OJO: puedes bajar n_seeds para pruebas rápidas (p.ej. 5)
    p_values = np.linspace(-0.5, 0.5, 11)  # [-0.5, -0.4, ..., 0.5]
    n_seeds = 30
    n_agents = base_cfg["general"]["N"]
    T = base_cfg["general"]["T"]

    all_results = []

    for regime in ["RE", "RH", "RC"]:
        print(f"\n=== Régimen: {regime} ===")
        df_reg = aggregate_over_seeds(
            base_cfg=base_cfg,
            regime=regime,
            p_values=p_values,
            n_seeds=n_seeds,
            n_agents=n_agents,
            T=T,
        )
        print(df_reg.head())
        all_results.append(df_reg)

    df_all = pd.concat(all_results, ignore_index=True)

    # Guardamos CSV con todo
    os.makedirs("outputs_srp_ev02c_global", exist_ok=True)
    csv_path = os.path.join("outputs_srp_ev02c_global", "srp_ev02c_results.csv")
    df_all.to_csv(csv_path, index=False)
    print(f"\nResultados agregados guardados en: {csv_path}")

    # Graficamos W(p) por régimen
    fig_path = os.path.join("outputs_srp_ev02c_global", "W_vs_p_EV02C.png")
    plot_W_vs_p(df_all, out_path=fig_path)
    print(f"Figura W_vs_p guardada en: {fig_path}")


if __name__ == "__main__":
    main()


=== Régimen: RE ===
  regime    p          W     G_eff     C_eff
0     RE -0.5  55.388333  0.033869  0.000872
1     RE -0.4  55.567500  0.034273  0.004248
2     RE -0.3  55.626667  0.034280  0.005327
3     RE -0.2  55.690000  0.034372  0.007287
4     RE -0.1  55.732500  0.034223  0.009666

=== Régimen: RH ===
  regime    p          W     G_eff     C_eff
0     RH -0.5  26.724167  0.146367  0.037362
1     RH -0.4  26.726667  0.146245  0.053201
2     RH -0.3  26.733333  0.147053  0.065696
3     RH -0.2  26.748333  0.148823  0.077910
4     RH -0.1  26.758333  0.150596  0.089204

=== Régimen: RC ===
  regime    p          W     G_eff     C_eff
0     RC -0.5  54.998333  0.486490  0.025955
1     RC -0.4  54.998333  0.489636  0.036720
2     RC -0.3  54.998333  0.491760  0.041510
3     RC -0.2  54.998333  0.493193  0.041698
4     RC -0.1  54.999167  0.498613  0.043776

Resultados agregados guardados en: outputs_srp_ev02c_global/srp_ev02c_results.csv
Figura W_vs_p guardada en: outputs_srp_ev02c

In [8]:
# srp_ev02c_payoff.py
#
# Analiza el payoff P_lambda(p) = G_eff(p) - lambda * C_eff(p)
# usando los resultados reales de EV-02C (APP-Fixed)
# guardados en: outputs_srp_ev02c_global/srp_ev02c_results.csv

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ruta al CSV que ya generaste
CSV_PATH = "outputs_srp_ev02c_global/srp_ev02c_results.csv"

def load_results(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    # Seguridad: quitamos filas raras
    df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=["G_eff", "C_eff", "W"])
    return df

def compute_payoff_grid(
    df: pd.DataFrame,
    lambdas = None,
):
    """
    df: DataFrame con columnas [regime, p, G_eff, C_eff]
    lambdas: lista o array de valores de lambda
    """
    if lambdas is None:
        lambdas = np.linspace(0.0, 1.0, 6)  # 0.0, 0.2, 0.4, 0.6, 0.8, 1.0

    results_rows = []

    regimes = sorted(df["regime"].unique())
    for reg in regimes:
        sub = df[df["regime"] == reg].copy()
        sub = sub.sort_values("p")
        for lam in lambdas:
            P = sub["G_eff"] - lam * sub["C_eff"]
            idx_max = int(P.values.argmax())
            p_star = float(sub["p"].iloc[idx_max])
            P_star = float(P.iloc[idx_max])

            results_rows.append(
                dict(
                    regime=reg,
                    lambda_=lam,
                    p_star=p_star,
                    P_star=P_star,
                )
            )

    df_payoff = pd.DataFrame(results_rows)
    return df_payoff

def plot_payoff_curves(df: pd.DataFrame, lambdas_to_plot=None, out_dir="outputs_srp_ev02c_global"):
    """
    Grafica P_lambda(p) para algunos valores de lambda, separado por régimen.
    """
    os.makedirs(out_dir, exist_ok=True)
    if lambdas_to_plot is None:
        lambdas_to_plot = [0.0, 0.2, 0.5, 1.0]

    regimes = sorted(df["regime"].unique())

    for reg in regimes:
        sub = df[df["regime"] == reg].copy().sort_values("p")
        p_vals = sub["p"].values
        G = sub["G_eff"].values
        C = sub["C_eff"].values

        plt.figure(figsize=(8, 5))
        for lam in lambdas_to_plot:
            P = G - lam * C
            plt.plot(p_vals, P, marker="o", label=f"λ = {lam:.2f}")

        plt.axvline(0.0, color="gray", linestyle="--", alpha=0.5)
        plt.xlabel("p (escala relativa de F_E)")
        plt.ylabel("P_λ(p) = G_eff(p) - λ C_eff(p)")
        plt.title(f"Payoff P_λ(p) para EV-02C real (regimen {reg})")
        plt.legend()
        plt.grid(True)

        fig_path = os.path.join(out_dir, f"Payoff_vs_p_EV02C_{reg}.png")
        plt.savefig(fig_path, dpi=150, bbox_inches="tight")
        plt.close()
        print(f"[{reg}] Figura guardada en: {fig_path}")

def main():
    # 1) Cargar resultados agregados EV-02C
    df = load_results(CSV_PATH)
    print("Resultados cargados:")
    print(df.head())

    # 2) Definir rango de lambdas
    lambdas = np.linspace(0.0, 1.0, 6)  # 0.0, 0.2, 0.4, 0.6, 0.8, 1.0

    # 3) Calcular p* para cada régimen y lambda
    df_payoff = compute_payoff_grid(df, lambdas=lambdas)
    print("\nÓptimos P_λ(p) por régimen:")
    print(df_payoff.sort_values(["regime", "lambda_"]))

    # 4) Guardar tabla
    out_dir = "outputs_srp_ev02c_global"
    os.makedirs(out_dir, exist_ok=True)
    payoff_csv = os.path.join(out_dir, "srp_ev02c_payoff_table.csv")
    df_payoff.to_csv(payoff_csv, index=False)
    print(f"\nTabla de payoff guardada en: {payoff_csv}")

    # 5) Graficar curvas P_λ(p)
    plot_payoff_curves(df, lambdas_to_plot=[0.0, 0.2, 0.5, 1.0], out_dir=out_dir)


if __name__ == "__main__":
    main()


Resultados cargados:
  regime    p          W     G_eff     C_eff
0     RE -0.5  55.388333  0.033869  0.000872
1     RE -0.4  55.567500  0.034273  0.004248
2     RE -0.3  55.626667  0.034280  0.005327
3     RE -0.2  55.690000  0.034372  0.007287
4     RE -0.1  55.732500  0.034223  0.009666

Óptimos P_λ(p) por régimen:
   regime  lambda_  p_star    P_star
0      RC      0.0     0.3  0.538159
1      RC      0.2     0.3  0.525342
2      RC      0.4     0.3  0.512526
3      RC      0.6     0.3  0.499709
4      RC      0.8     0.3  0.486893
5      RC      1.0     0.3  0.474076
6      RE      0.0    -0.2  0.034372
7      RE      0.2    -0.5  0.033695
8      RE      0.4    -0.5  0.033520
9      RE      0.6    -0.5  0.033346
10     RE      0.8    -0.5  0.033172
11     RE      1.0    -0.5  0.032997
12     RH      0.0     0.5  0.158980
13     RH      0.2    -0.5  0.138894
14     RH      0.4    -0.5  0.131422
15     RH      0.6    -0.5  0.123949
16     RH      0.8    -0.5  0.116477
17     RH     